In [ ]:
%pip install llama-index llama-index-llms-mistralai llama-index-embeddings-fastembed -q
%pip install gradio -q

In [ ]:
!git clone https://github.com/docty/content_retrieval.git

In [ ]:
import os
import getpass

os.environ["MISTRAL_API_KEY"] = getpass.getpass('Api Key')

In [ ]:
from content_retrieval.contextaware import LLMContextAware
from content_retrieval.utils import download_data

In [ ]:
#https://lmu.edu.ng/XownCMS/cv/FALADE_SUNDAY_A.%20CV.pdf
download_data(data_url='https://www.princexml.com/samples/invoice/invoicesample.pdf')

In [ ]:
import gradio as gr
import time

llmContext = LLMContextAware()
custom_docs = llmContext.load_data('./data/sample.pdf')
query_engine = llmContext.query_engine(custom_docs)

 
def user(user_message, history: list):
    history = history + [{"role": "user", "content": user_message}]
    return "", history

 
def bot(history: list):
    try:
        user_message = history[-1]['content']
        full_response = query_engine.query(user_message).response
    except Exception as e:
        full_response = f"Error: {e}"
    
    history.append({"role": "assistant", "content": ""})
    
    for char in full_response:
        history[-1]['content'] += char
        time.sleep(0.02)  
        yield history


with gr.Blocks(title="Custom Chatbot") as demo:
    gr.Markdown("## Ask questions about the document.")
    
    chatbot = gr.Chatbot(type="messages")
    msg = gr.Textbox(label="Your question", placeholder="Ask something about the document...")
    clear = gr.Button("Clear")

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    clear.click(lambda: None, None, chatbot, queue=False)


    demo.launch(share=True)
